In [1]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

import numpy as np
from sklearn import preprocessing, metrics, model_selection 
import pandas as pd

In [2]:
df = pd.read_csv('../Datos/Iris.csv')

# Tomamos todas las columnas menos la última
X = np.array(df.iloc[:, 0:-1])
Y = np.array(df.iloc[:,-1])
encoder = preprocessing.LabelEncoder()
Y = encoder.fit_transform(Y)

In [3]:
#--- CONJUNTOS DE ENTRENAMIENTO Y TESTEO ---
X_train, X_test, Y_train, Y_test = model_selection.train_test_split( \
        X, Y, test_size=0.20) #, random_state=42)

Y_trainB = to_categorical(Y_train)

In [4]:
normalizarEntrada = 1  # 1 si normaliza; 0 si no

if normalizarEntrada:
    # Escala los valores entre 0 y 1
    min_max_scaler = preprocessing.MinMaxScaler()
    X_train = min_max_scaler.fit_transform(X_train)
    X_test = min_max_scaler.transform(X_test)

### Definición del modelo

In [5]:
# Crear un modelo de capas secuenciales
model=Sequential()

# Agregar las capas al modelo
model.add(Dense(2, input_shape=[4], activation='tanh'))
model.add(Dense(3, activation='softmax'))

# Imprimir un resumen del modelo
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 2)                 10        
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 9         
Total params: 19
Trainable params: 19
Non-trainable params: 0
_________________________________________________________________


In [6]:
# Configuración para entrenamiento
#model.compile(optimizer='sgd', loss='mse', metrics='accuracy')
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics='accuracy')


In [7]:
# Reservamos 30 muestras para validacion --
X_val = X_train[-30:]   # últimas 30 filas de X_train
Y_valB = Y_trainB[-30:]

X_train = X_train[:-30]  # todas las filas menos las últimas 30
Y_trainB = Y_trainB[:-30]

In [8]:
es = EarlyStopping(monitor='val_accuracy', patience=30, min_delta=0.0001)

H = model.fit(x = X_train, y = Y_trainB, epochs=4000, batch_size = 20, 
              validation_data = (X_val, Y_valB),  callbacks=[es])

Epoch 1/4000
5/5 [==============================] - 0s 51ms/step - loss: 1.0149 - accuracy: 0.4000 - val_loss: 1.0421 - val_accuracy: 0.3333
Epoch 2/4000
5/5 [==============================] - 0s 9ms/step - loss: 1.0133 - accuracy: 0.4111 - val_loss: 1.0401 - val_accuracy: 0.3333
Epoch 3/4000
5/5 [==============================] - 0s 8ms/step - loss: 1.0115 - accuracy: 0.4444 - val_loss: 1.0381 - val_accuracy: 0.3333
Epoch 4/4000
5/5 [==============================] - 0s 9ms/step - loss: 1.0099 - accuracy: 0.4444 - val_loss: 1.0365 - val_accuracy: 0.3667
Epoch 5/4000
5/5 [==============================] - 0s 8ms/step - loss: 1.0084 - accuracy: 0.4667 - val_loss: 1.0345 - val_accuracy: 0.4000
Epoch 6/4000
5/5 [==============================] - 0s 9ms/step - loss: 1.0069 - accuracy: 0.5000 - val_loss: 1.0328 - val_accuracy: 0.4333
Epoch 7/4000
5/5 [==============================] - 0s 8ms/step - loss: 1.0053 - accuracy: 0.5000 - val_loss: 1.0308 - val_accuracy: 0.4667
Epoch 8/4000
5/5 [=

5/5 [==============================] - 0s 6ms/step - loss: 0.8991 - accuracy: 0.6778 - val_loss: 0.9135 - val_accuracy: 0.7000
Epoch 60/4000
5/5 [==============================] - 0s 3ms/step - loss: 0.8965 - accuracy: 0.6778 - val_loss: 0.9108 - val_accuracy: 0.7000
Epoch 61/4000
5/5 [==============================] - 0s 3ms/step - loss: 0.8940 - accuracy: 0.6778 - val_loss: 0.9082 - val_accuracy: 0.7000
Epoch 62/4000
5/5 [==============================] - 0s 3ms/step - loss: 0.8917 - accuracy: 0.6778 - val_loss: 0.9055 - val_accuracy: 0.7000
Epoch 63/4000
5/5 [==============================] - 0s 4ms/step - loss: 0.8890 - accuracy: 0.6778 - val_loss: 0.9029 - val_accuracy: 0.7000
Epoch 64/4000
5/5 [==============================] - 0s 6ms/step - loss: 0.8867 - accuracy: 0.6778 - val_loss: 0.9003 - val_accuracy: 0.7000
Epoch 65/4000
5/5 [==============================] - 0s 3ms/step - loss: 0.8842 - accuracy: 0.6778 - val_loss: 0.8978 - val_accuracy: 0.7000
Epoch 66/4000
5/5 [========

In [9]:
print("épocas = %d" % len(H.history["val_accuracy"]))   
print("Epocas = %d" % es.stopped_epoch)

épocas = 73
Epocas = 72


In [10]:
# predecir la salida del modelo
Y_pred = model.predict(X_train)
Y_pred2 = np.argmax(Y_pred,axis=1)

Y_train = Y_train[:-30]   # usar la misma cantidad que los empleados en el entrenamiento

print("%% aciertos X_train : %.3f" % metrics.accuracy_score(Y_train, Y_pred2))

% aciertos X_train : 0.678


In [11]:
report = metrics.classification_report(Y_train, Y_pred2)
print("Confusion matrix Training:\n%s" % report) 

MM = metrics.confusion_matrix(Y_train, Y_pred2)
print("Confusion matrix:\n%s" % MM)

Confusion matrix Training:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        28
           1       1.00      0.03      0.06        30
           2       0.52      1.00      0.69        32

    accuracy                           0.68        90
   macro avg       0.84      0.68      0.58        90
weighted avg       0.83      0.68      0.58        90

Confusion matrix:
[[28  0  0]
 [ 0  1 29]
 [ 0  0 32]]
